**PREGUNTAS CASO NEGOCIO SPARK**

Para la resolución de los ejercicios del Caso de Negocio, se partirá de las tablas iniciales que se han guardado en Hive al comienzo de la práctica resuelta

In [0]:
# Se realizan todos los imports necesarios
from pyspark.sql.functions import * 
from pyspark.sql import functions  as f
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pandas as pd
import datetime 
import calendar
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType
from pyspark.sql import SparkSession

In [0]:
dbutils.fs.rm("dbfs:/user/hive/warehouse/df_clientes", recurse=True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/df_ofertas", recurse=True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/df_facturas_mes_ant", recurse=True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/df_facturas_mes_actual", recurse=True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/df_consumos_diarios", recurse=True)
dbutils.fs.rm("df_clientes_edad", recurse=True)
dbutils.fs.rm("df_7", recurse=True)



Out[3]: False

In [0]:
%scala
dbutils.fs.rm("dbfs:/user/hive/warehouse/df_clientes_edades", true)
dbutils.fs.rm("dbfs:/user/hive/warehouse/df_7", true)


res1: Boolean = true

In [0]:
#clientes
columns = ['id_cliente','nombre','edad','sexo','provincia','pais']
data = [(1,'Pablo Perez',26,'M','Madrid','España'),(2,'Eduardo Redondo',58,'M','Bogota','Colombia'),(3,'Roberto Salazar',68,'M','Monterrey','Mexico'),
        (4,'Pedro Conde',32,'M','Madrid','España'),(5,'Ana Robles',41,'F','Valladolid','España'),(6,'David Roldan',74,'M','Guadalajara','Mexico'),
        (7,'Carmen Arauzo',19,'F','Medellin','Colombia'),(8,'Silvia Rodriguez',49,'F','Barcelona','España'),(9,'David Cardoso',24,'M','Cali','Colombia'),
        (10,'Pablo Lopez',40,'M','Sevilla','España'),(11,'Luis Rodrigues',20,'M','Bogota','Colombia'),(12,'Alvaro Monroy',75,'M','Monterrey','Mexico'),
        (13,'Victor Ruiz',53,'M','Valladolid','España'),(14,'Melisa Aguado',30,'F','Bogota','Colombia'),(15,'Cristian Cuadrado',52,'M','Guadalajara','Mexico'),
        (16,'Cristina Sanz',49,'F','Madrid','España'),(17,'Jorge Recio',18,'M','Valladolid','España'),(18,'Laura Luiz',44,'F','Cali','Colombia'),
        (19,'Juan Carlos Iglesias',38,'M','Barcelona','España'),(20,'Oscar Rico',22,'M','Valladolid','España'),(21,'Fatima Cuevas',29,'F','Cali','Colombia'),
        (22,'Clara Suarez',21,'F','Sevilla','España'),(23,'Fernanda Gomez',78,'F','Monterrey','Mexico'),(24,'Ruben Garcia',68,'M','Sevilla','España'),
        (25,'Ines Barcero',29,'F','Guadalajara','Mexico'),(26,'Celia Castro',47,'F','Barcelona','España'),(27,'Roberta Varado',64,'F','Cali','Colombia'),
        (28,'Walter Ramos',54,'M','Cali','Colombia'),(29,'Romina Verde',29,'F','Guadalajara','Mexico'),(30,'Marta Rodrigo',67,'F','Guadalajara','Mexico')]
        
        #(25,'Roberto Versero',66,'Sevilla','M','España'),(26,'Javier Ramos',36,'M','Madrid','España'),(27,'Ana Bolana',32,'F','Barcelona','España'),
        #(28,'Lucia Alonso',23,'F','Valladolid','España'),(29,'Elena Luengo',27,'F','Valencia','España'),(30,'Rebeca Velez',69,'F','Cali','Colombia'),]

df_clientes = spark.createDataFrame(data, columns)
df_clientes.write.mode("overwrite").option("encoding", "UTF-8").saveAsTable('df_clientes')
# display(df_clientes)
# df_clientes.show(n=30)




In [0]:
# Se presentan 10 tipos de ofertas diferentes (combinaciones entre Fibra Optica, Fijo, Movil (llamadas o mensajes ilimitados), Television)
#ofertas estandar
columns = ['id_oferta','descripcion','importe'] 
data = [(1,'Fibra Optica 300MB + Fijo ilimitado + Television',70.00),
        (2,'Fibra Optica 600MB + Fijo ilimitado + Television', 78.50),
        (3,'Fibra Optica 300MB + Movil llamadas y datos ilimitados + Television', 85.00),
        (4,'Fibra Optica 600MB + Movil llamadas y datos ilimitados + Television', 95.00),
        (5,'Fibra Optica 300MB + Fijo ilimitado + Movil llamadas y datos ilimitados', 102.50),
        (6,'Fibra Optica 600MB + Fijo ilimitado + Movil llamadas y datos ilimitados + Television', 124.50),
        (7,'Fibra Optica 300MB + Movil llamadas ilimitadas + Television', 80.50),
        (8,'Fibra Optica 300MB + Movil datos ilimitados + Television', 84.00),
        (9,'Fijo llamadas ilimitadas + Movil llamadas y datos ilimitados', 42.50),
        (10,'Fijo llamadas ilimitadas + Movil llamadas ilimitadas', 31.99)]

df_ofertas = spark.createDataFrame(data, columns)

# display(df_ofertas)
# df_ofertas.show(n=10, truncate=False)
df_ofertas.write.mode("overwrite").option("encoding", "UTF-8").saveAsTable('df_ofertas')


In [0]:
#facturas mes ant
columns = ['id_cliente','id_oferta','importe','fecha'] 
data = [(1,6,124.5,"2020-07-31"),(1,6,118.60,"2020-07-31"),(1,9,36.5,"2020-07-31"),
        (2,4,95.0,"2020-07-31"),(3,5,102.5,"2020-07-31"),(5,7,80.5,"2020-07-31"),
        (5,9,40.5,"2020-07-31"),(6,8,84.0,"2020-07-31"),(7,4,95.0,"2020-07-31"),
        (8,5,102.5,"2020-07-31"),(10,6,124.5,"2020-07-31"),(11,3,80.0,"2020-07-31"),
        (12,1,70.0,"2020-07-31"),(13,4,95.0,"2020-07-31"),(14,9,42.5,"2020-07-31"),
        (15,2,78.50,"2020-07-31"),(15,6,115.0,"2020-07-31"),(16,1,70.0,"2020-07-31"),
        (17,8,84.0,"2020-07-31"),(18,5,102.5,"2020-07-31"),(18,9,37.5,"2020-07-31"),
        (20,4,95.0,"2020-07-31"),(22,2,78.5,"2020-07-31"),(22,7,69.5,"2020-07-31"),
        (23,3,85.0,"2020-07-31"),(24,3,85.0,"2020-07-31"),(25,9,42.5,"2020-07-31"),
        (25,10,31.99,"2020-07-31"),(26,4,95.0,"2020-07-31"),(27,8,84.0,"2020-07-31"),
        (27,9,41.5,"2020-07-31"),(29,6,124.5,"2020-07-31"),(30,1,70.0,"2020-07-31"),
        (30,2,70.0,"2020-07-31")]
        
df_facturas_mes_ant = spark.createDataFrame(data, columns)
# display(df_facturas_mes_ant)
# df_facturas_mes_ant.show(n=40, truncate=False)
df_facturas_mes_ant.write.mode("overwrite").option("encoding", "UTF-8").saveAsTable('df_facturas_mes_ant')

In [0]:
#factura actual
# Se dan de baja en el mes actual: 1y,27(solo de uno de los contratos),16,20,24
# cliente 25 rebaja por fidelidad
columns = ['id_cliente','id_oferta','importe','fecha'] 
data = [(1,6,124.5,"2020-08-31"),(1,6,118.60,"2020-08-31"),(2,4,95.0,"2020-08-31"),
        (3,5,102.5,"2020-08-31"),(4,1,70.0,"2020-08-31"),(5,7,80.5,"2020-08-31"),
        (5,9,40.5,"2020-08-31"),(6,8,84.0,"2020-08-31"),(7,4,95.0,"2020-08-31"),
        (8,5,100.5,"2020-08-31"),(9,7,78.50,"2020-08-31"),(9,10,29.99,"2020-08-31"),
        (10,6,124.5,"2020-08-31"),(11,3,83.0,"2020-08-31"),(12,1,70.0,"2020-08-31"),
        (13,4,95.0,"2020-08-31"),(14,9,42.5,"2020-08-31"),(15,2,78.50,"2020-08-31"),
        (15,6,115.0,"2020-08-31"),(17,8,84.0,"2020-08-31"),(18,5,102.5,"2020-08-31"),
        (18,9,39.0,"2020-08-31"),(19,3,85.0,"2020-08-31"),(21,6,124.50,"2020-08-31"),
        (22,2,78.5,"2020-08-31"),(22,7,72.5,"2020-08-31"),(23,3,85.0,"2020-08-31"),
        (25,9,42.5,"2020-08-31"),(25,10,29.70,"2020-08-31"),(26,4,95.0,"2020-08-31"),
        (27,8,84.0,"2020-08-31"),(28,5,102.50,"2020-08-31"),(29,6,121.50,"2020-08-31"),
        (30,1,70.0,"2020-08-31"),(30,2,71.0,"2020-08-31")]

df_facturas_mes_actual = spark.createDataFrame(data, columns)
# display(df_facturas_mes_actual)
# df_facturas_mes_actual.show(n=40, truncate=False)
df_facturas_mes_actual.write.mode("overwrite").option("encoding", "UTF-8").saveAsTable('df_facturas_mes_actual')

In [0]:
#consumos diarios
columns = ['id_cliente','consumo_datos_MB','sms_enviados','minutos_llamadas_movil','minutos_llamadas_fijo','fecha']
data = [(1,664,3,25,0,"2020-08-01"),(1,943,0,12,5,"2020-08-02"),(1,1035,1,46,10,"2020-08-03"),(1,760,3,17,0,"2020-08-04"),
        (1,1409,1,31,4,"2020-08-05"),(1,1627,0,20,0,"2020-08-06"),(1,739,3,21,0,"2020-08-07"),(1,760,0,12,0,"2020-08-08"),
        (1,928,0,0,24,"2020-08-09"),(1,829,2,46,0,"2020-08-10"),(1,1810,3,0,0,"2020-08-11"),(1,760,0,25,0,"2020-08-12"),
        (1,603,0,41,0,"2020-08-13"),(1,732,0,34,102,"2020-08-14"),(1,729,1,0,50,"2020-08-15"),(1,760,1,31,0,"2020-08-16"),
        (1,1934,2,78,0,"2020-08-17"),(1,827,0,29,32,"2020-08-18"),(1,691,0,0,7,"2020-08-19"),(1,760,2,19,0,"2020-08-20"),
        (1,1142,5,14,0,"2020-08-21"),(1,1242,1,40,0,"2020-08-22"),(1,830,0,32,2,"2020-08-23"),(1,760,4,56,17,"2020-08-24"),
        (1,734,0,19,0,"2020-08-25"),(1,1729,0,11,0,"2020-08-26"),(1,819,1,95,18,"2020-08-27"),(1,760,0,60,12,"2020-08-28"),
        (1,302,0,29,0,"2020-08-29"),(1,497,3,14,0,"2020-08-30"),(1,1281,2,21,0,"2020-08-31"),
        (2,321,7,38,13,"2020-08-01"),(2,145,0,8,0,"2020-08-02"),(2,420,1,46,4,"2020-08-03"),(2,492,1,9,0,"2020-08-04"),
        (2,264,8,29,31,"2020-08-05"),(2,172,10,22,0,"2020-08-06"),(2,314,2,74,34,"2020-08-07"),(2,312,2,0,16,"2020-08-08"),
        (2,312,11,32,27,"2020-08-09"),(2,243,2,48,0,"2020-08-10"),(2,423,4,38,12,"2020-08-11"),(2,492,1,10,22,"2020-08-12"),
        (2,492,12,29,29,"2020-08-13"),(2,274,4,60,0,"2020-08-14"),(2,402,0,28,20,"2020-08-15"),(2,471,0,12,30,"2020-08-16"),
        (2,462,1,36,32,"2020-08-17"),(2,294,3,78,42,"2020-08-18"),(2,593,0,29,65,"2020-08-19"),(2,380,12,53,9,"2020-08-20"),
        (2,568,2,42,20,"2020-08-21"),(2,300,2,40,35,"2020-08-22"),(2,600,0,35,29,"2020-08-23"),(2,397,9,37,40,"2020-08-24"),
        (2,623,4,59,42,"2020-08-25"),(2,302,1,34,15,"2020-08-26"),(2,319,1,52,34,"2020-08-27"),(2,901,8,28,83,"2020-08-28"),
        (2,611,0,41,38,"2020-08-29"),(2,319,0,21,19,"2020-08-30"),(2,387,3,19,18,"2020-08-31"),
        (3,123,5,42,14,"2020-08-01"),(3,460,4,24,16,"2020-08-02"),(3,291,3,27,18,"2020-08-03"),(3,149,7,20,0,"2020-08-04"),
        (3,182,6,20,19,"2020-08-05"),(3,245,12,28,21,"2020-08-06"),(3,302,2,31,26,"2020-08-07"),(3,200,8,18,8,"2020-08-08"),
        (3,239,7,40,25,"2020-08-09"),(3,111,9,29,0,"2020-08-10"),(3,191,1,14,29,"2020-08-11"),(3,28,12,4,10,"2020-08-12"),
        (3,201,3,15,32,"2020-08-13"),(3,102,8,31,0,"2020-08-14"),(3,62,10,29,31,"2020-08-15"),(3,82,2,49,42,"2020-08-16"),
        (3,72,1,0,15,"2020-08-17"),(3,94,2,12,18,"2020-08-18"),(3,30,11,20,45,"2020-08-19"),(3,172,1,63,8,"2020-08-20"),
        (3,80,1,12,30,"2020-08-21"),(3,274,7,5,11,"2020-08-22"),(3,219,7,8,15,"2020-08-23"),(3,104,7,6,2,"2020-08-24"),
        (3,180,9,48,45,"2020-08-25"),(3,210,0,10,44,"2020-08-26"),(3,310,2,17,18,"2020-08-27"),(3,182,9,0,1,"2020-08-28"),
        (3,164,0,32,5,"2020-08-29"),(3,289,0,12,33,"2020-08-30"),(3,126,3,23,22,"2020-08-31"),
        (4,523,0,87,0,"2020-08-01"),(4,628,1,56,0,"2020-08-02"),(4,1021,0,15,0,"2020-08-03"),(4,610,1,45,0,"2020-08-04"),
        (4,612,0,65,0,"2020-08-05"),(4,359,0,102,15,"2020-08-06"),(4,832,1,62,0,"2020-08-07"),(4,583,0,30,0,"2020-08-08"),
        (4,490,2,81,20,"2020-08-09"),(4,479,0,81,0,"2020-08-10"),(4,792,2,19,0,"2020-08-11"),(4,1216,0,27,32,"2020-08-12"),
        (4,512,0,32,0,"2020-08-13"),(4,824,4,76,0,"2020-08-14"),(4,682,1,37,18,"2020-08-15"),(4,829,2,47,0,"2020-08-16"),
        (4,921,0,45,0,"2020-08-17"),(4,729,0,42,0,"2020-08-18"),(4,534,2,95,64,"2020-08-19"),(4,967,3,51,82,"2020-08-20"),
        (4,721,3,54,0,"2020-08-21"),(4,218,0,38,17,"2020-08-22"),(4,729,0,85,0,"2020-08-23"),(4,826,1,38,0,"2020-08-24"),
        (4,845,1,34,0,"2020-08-25"),(4,367,2,71,21,"2020-08-26"),(4,502,0,60,0,"2020-08-27"),(4,800,0,25,0,"2020-08-28"),
        (4,629,0,49,0,"2020-08-29"),(4,588,0,50,0,"2020-08-30"),(4,693,1,31,40,"2020-08-31"),
        (5,241,1,125,20,"2020-08-01"),(5,315,0,36,32,"2020-08-02"),(5,301,1,20,80,"2020-08-03"),(5,152,5,38,62,"2020-08-04"),
        (5,289,4,110,45,"2020-08-05"),(5,510,2,74,40,"2020-08-06"),(5,178,2,19,67,"2020-08-07"),(5,241,3,41,59,"2020-08-08"),
        (5,301,6,62,39,"2020-08-09"),(5,86,4,82,47,"2020-08-10"),(5,201,6,75,61,"2020-08-11"),(5,358,8,51,62,"2020-08-12"),
        (5,176,8,45,0,"2020-08-13"),(5,109,5,102,12,"2020-08-14"),(5,798,5,91,45,"2020-08-15"),(5,301,0,43,71,"2020-08-16"),
        (5,198,9,73,70,"2020-08-17"),(5,125,2,115,17,"2020-08-18"),(5,182,4,99,49,"2020-08-19"),(5,222,0,30,80,"2020-08-20"),
        (5,234,3,82,34,"2020-08-21"),(5,201,3,70,20,"2020-08-22"),(5,283,3,130,27,"2020-08-23"),(5,333,2,28,0,"2020-08-24"),
        (5,387,5,90,41,"2020-08-25"),(5,258,5,62,30,"2020-08-26"),(5,251,7,100,36,"2020-08-27"),(5,475,6,79,16,"2020-08-28"),
        (5,312,12,15,80,"2020-08-29"),(5,328,1,50,19,"2020-08-30"),(5,291,11,32,65,"2020-08-31"),
        (6,50,7,10,50,"2020-08-01"),(6,80,6,10,0,"2020-08-02"),(6,65,12,0,40,"2020-08-03"),(6,284,14,0,42,"2020-08-04"),
        (6,20,9,0,46,"2020-08-05"),(6,36,4,0,67,"2020-08-06"),(6,90,9,10,36,"2020-08-07"),(6,201,18,0,83,"2020-08-08"),
        (6,28,13,0,64,"2020-08-09"),(6,150,7,22,46,"2020-08-10"),(6,135,7,0,65,"2020-08-11"),(6,196,22,17,68,"2020-08-12"),
        (6,49,15,0,37,"2020-08-13"),(6,0,0,0,92,"2020-08-14"),(6,122,6,0,56,"2020-08-15"),(6,146,3,0,94,"2020-08-16"),
        (6,102,13,15,48,"2020-08-17"),(6,38,8,0,17,"2020-08-18"),(6,180,11,14,41,"2020-08-19"),(6,159,6,0,26,"2020-08-20"),
        (6,27,16,24,50,"2020-08-21"),(6,156,12,0,75,"2020-08-22"),(6,193,10,20,39,"2020-08-23"),(6,478,4,30,64,"2020-08-24"),
        (6,35,5,0,80,"2020-08-25"),(6,120,15,28,52,"2020-08-26"),(6,201,8,0,54,"2020-08-27"),(6,310,1,28,15,"2020-08-28"),
        (6,40,3,8,35,"2020-08-29"),(6,73,4,16,34,"2020-08-30"),(6,174,7,25,13,"2020-08-31"),
        (7,1456,0,37,0,"2020-08-01"),(7,946,0,24,24,"2020-08-02"),(7,1302,0,72,0,"2020-08-03"),(7,1356,0,13,45,"2020-08-04"),
        (7,1256,1,28,0,"2020-08-05"),(7,985,0,11,5,"2020-08-06"),(7,934,0,40,0,"2020-08-07"),(7,464,1,65,0,"2020-08-08"),
        (7,930,0,0,0,"2020-08-09"),(7,1385,0,30,0,"2020-08-10"),(7,635,0,38,0,"2020-08-11"),(7,588,2,60,0,"2020-08-12"),
        (7,849,2,40,0,"2020-08-13"),(7,1405,3,10,0,"2020-08-14"),(7,911,2,67,24,"2020-08-15"),(7,1495,0,83,0,"2020-08-16"),
        (7,830,0,47,23,"2020-08-17"),(7,1295,0,52,0,"2020-08-18"),(7,844,1,9,0,"2020-08-19"),(7,1246,0,12,4,"2020-08-20"),
        (7,1044,0,28,0,"2020-08-21"),(7,732,1,0,0,"2020-08-22"),(7,1468,4,13,0,"2020-08-23"),(7,967,0,79,0,"2020-08-24"),
        (7,1953,1,17,19,"2020-08-25"),(7,802,0,24,11,"2020-08-26"),(7,1853,1,71,15,"2020-08-27"),(7,1344,0,61,16,"2020-08-28"),
        (7,1067,2,13,0,"2020-08-29"),(7,1740,0,33,57,"2020-08-30"),(7,1122,0,38,12,"2020-08-31"),
        (8,346,5,75,30,"2020-08-01"),(8,502,6,74,40,"2020-08-02"),(8,482,1,12,62,"2020-08-03"),(8,200,10,63,69,"2020-08-04"),
        (8,386,6,83,29,"2020-08-05"),(8,310,3,62,28,"2020-08-06"),(8,500,3,123,0,"2020-08-07"),(8,284,5,71,31,"2020-08-08"),
        (8,305,12,42,19,"2020-08-09"),(8,375,13,80,15,"2020-08-10"),(8,319,5,84,29,"2020-08-11"),(8,249,4,42,53,"2020-08-12"),
        (8,401,10,26,0,"2020-08-13"),(8,281,15,37,76,"2020-08-14"),(8,318,7,86,34,"2020-08-15"),(8,394,8,39,12,"2020-08-16"),
        (8,250,8,56,63,"2020-08-17"),(8,272,4,42,59,"2020-08-18"),(8,492,1,70,46,"2020-08-19"),(8,365,3,51,33,"2020-08-20"),
        (8,284,2,12,25,"2020-08-21"),(8,266,5,50,15,"2020-08-22"),(8,172,1,67,12,"2020-08-23"),(8,402,0,68,8,"2020-08-24"),
        (8,265,4,103,31,"2020-08-25"),(8,388,9,62,43,"2020-08-26"),(8,193,1,48,43,"2020-08-27"),(8,95,6,77,18,"2020-08-28"),
        (8,192,0,93,22,"2020-08-29"),(8,293,12,71,55,"2020-08-30"),(8,174,1,52,36,"2020-08-31"),
        (9,1304,0,13,0,"2020-08-01"),(9,720,0,11,5,"2020-08-02"),(9,1111,1,8,12,"2020-08-03"),(9,1760,1,5,1,"2020-08-04"),
        (9,1502,1,0,3,"2020-08-05"),(9,1455,2,21,0,"2020-08-06"),(9,850,2,7,0,"2020-08-07"),(9,799,2,7,7,"2020-08-08"),
        (9,623,0,5,0,"2020-08-09"),(9,1294,1,5,0,"2020-08-10"),(9,950,0,4,0,"2020-08-11"),(9,923,0,15,0,"2020-08-12"),
        (9,730,0,8,0,"2020-08-13"),(9,823,0,0,0,"2020-08-14"),(9,957,0,0,7,"2020-08-15"),(9,829,0,0,2,"2020-08-16"),
        (9,945,0,0,5,"2020-08-17"),(9,392,0,0,3,"2020-08-18"),(9,1028,0,3,12,"2020-08-19"),(9,729,0,0,0,"2020-08-20"),
        (9,920,4,4,0,"2020-08-21"),(9,720,1,2,9,"2020-08-22"),(9,1932,0,8,27,"2020-08-23"),(9,999,3,0,6,"2020-08-24"),
        (9,869,0,8,0,"2020-08-25"),(9,1830,0,20,15,"2020-08-26"),(9,1234,1,0,0,"2020-08-27"),(9,1885,4,24,13,"2020-08-28"),
        (9,888,2,13,0,"2020-08-29"),(9,929,1,7,6,"2020-08-30"),(9,606,2,37,0,"2020-08-31"),
        (10,482,3,35,20,"2020-08-01"),(10,418,0,32,18,"2020-08-02"),(10,382,3,22,24,"2020-08-03"),(10,212,2,30,16,"2020-08-04"),
        (10,510,7,43,6,"2020-08-05"),(10,399,1,43,25,"2020-08-06"),(10,593,0,23,25,"2020-08-07"),(10,392,2,4,25,"2020-08-08"),
        (10,692,0,13,17,"2020-08-09"),(10,312,0,2,15,"2020-08-10"),(10,492,0,13,27,"2020-08-11"),(10,433,0,32,18,"2020-08-12"),
        (10,582,2,0,29,"2020-08-13"),(10,422,3,31,6,"2020-08-14"),(10,598,1,4,46,"2020-08-15"),(10,510,0,50,12,"2020-08-16"),
        (10,382,5,34,0,"2020-08-17"),(10,628,0,0,21,"2020-08-18"),(10,428,4,22,37,"2020-08-19"),(10,413,1,12,34,"2020-08-20"),
        (10,318,6,6,7,"2020-08-21"),(10,700,2,16,30,"2020-08-22"),(10,1021,5,11,32,"2020-08-23"),(10,284,8,0,28,"2020-08-24"),
        (10,492,2,3,6,"2020-08-25"),(10,238,0,21,16,"2020-08-26"),(10,453,0,0,17,"2020-08-27"),(10,333,10,6,14,"2020-08-28"),
        (10,609,0,14,16,"2020-08-29"),(10,395,5,0,7,"2020-08-30"),(10,530,1,9,0,"2020-08-31"),
        (11,2134,0,8,11,"2020-08-01"),(11,1093,0,18,12,"2020-08-02"),(11,1467,0,15,2,"2020-08-03"),(11,1760,0,4,9,"2020-08-04"),
        (11,1482,0,12,0,"2020-08-05"),(11,1004,0,0,2,"2020-08-06"),(11,2402,1,0,8,"2020-08-07"),(11,838,1,0,0,"2020-08-08"),
        (11,922,0,0,0,"2020-08-09"),(11,983,2,2,3,"2020-08-10"),(11,1803,1,7,10,"2020-08-11"),(11,927,0,12,4,"2020-08-12"),
        (11,1450,1,16,3,"2020-08-13"),(11,889,0,7,9,"2020-08-14"),(11,1234,0,28,0,"2020-08-15"),(11,1583,0,17,0,"2020-08-16"),
        (11,1503,0,0,0,"2020-08-17"),(11,798,1,0,3,"2020-08-18"),(11,1652,0,0,0,"2020-08-19"),(11,1362,0,0,6,"2020-08-20"),
        (11,1793,0,0,0,"2020-08-21"),(11,1928,0,0,0,"2020-08-22"),(11,821,0,7,11,"2020-08-23"),(11,1278,1,0,0,"2020-08-24"),
        (11,1112,0,7,18,"2020-08-25"),(11,1356,0,19,4,"2020-08-26"),(11,629,2,15,0,"2020-08-27"),(11,1029,2,8,13,"2020-08-28"),
        (11,829,0,4,8,"2020-08-29"),(11,1224,3,0,0,"2020-08-30"),(11,492,0,6,8,"2020-08-31"),
        (12,92,5,32,30,"2020-08-01"),(12,201,1,21,17,"2020-08-02"),(12,218,12,8,13,"2020-08-03"),(12,49,2,21,4,"2020-08-04"),
        (12,134,8,16,40,"2020-08-05"),(12,192,8,19,23,"2020-08-06"),(12,167,15,9,6,"2020-08-07"),(12,92,12,24,16,"2020-08-08"),
        (12,27,1,39,21,"2020-08-09"),(12,78,17,27,15,"2020-08-10"),(12,51,8,17,8,"2020-08-11"),(12,106,14,12,13,"2020-08-12"),
        (12,39,5,27,72,"2020-08-13"),(12,90,4,18,31,"2020-08-14"),(12,73,9,10,0,"2020-08-15"),(12,176,12,9,7,"2020-08-16"),
        (12,182,0,17,52,"2020-08-17"),(12,91,8,12,8,"2020-08-18"),(12,82,7,0,6,"2020-08-19"),(12,234,6,0,15,"2020-08-20"),
        (12,76,9,10,29,"2020-08-21"),(12,108,6,18,8,"2020-08-22"),(12,72,9,14,25,"2020-08-23"),(12,188,2,7,8,"2020-08-24"),
        (12,111,13,11,17,"2020-08-25"),(12,115,0,7,9,"2020-08-26"),(12,63,6,30,4,"2020-08-27"),(12,177,0,17,22,"2020-08-28"),
        (12,124,12,20,28,"2020-08-29"),(12,100,3,0,13,"2020-08-30"),(12,94,14,19,13,"2020-08-31"),  
        (13,530,5,15,12,"2020-08-01"),(13,736,4,0,0,"2020-08-02"),(13,649,4,13,22,"2020-08-03"),(13,698,6,0,0,"2020-08-04"),
        (13,721,2,17,23,"2020-08-05"),(13,816,2,18,6,"2020-08-06"),(13,701,3,19,18,"2020-08-07"),(13,726,7,0,17,"2020-08-08"),
        (13,394,3,9,18,"2020-08-09"),(13,793,0,9,2,"2020-08-10"),(13,612,9,0,5,"2020-08-11"),(13,746,2,14,8,"2020-08-12"),
        (13,455,1,0,14,"2020-08-13"),(13,439,1,21,7,"2020-08-14"),(13,682,8,7,9,"2020-08-15"),(13,893,0,19,23,"2020-08-16"),
        (13,801,0,13,0,"2020-08-17"),(13,399,7,2,15,"2020-08-18"),(13,723,0,16,15,"2020-08-19"),(13,487,0,21,22,"2020-08-20"),
        (13,739,6,20,8,"2020-08-21"),(13,465,6,3,5,"2020-08-22"),(13,788,3,7,13,"2020-08-23"),(13,589,5,12,6,"2020-08-24"),
        (13,666,2,13,3,"2020-08-25"),(13,582,0,0,16,"2020-08-26"),(13,625,7,12,12,"2020-08-27"),(13,501,3,4,8,"2020-08-28"),
        (13,615,0,9,14,"2020-08-29"),(13,611,3,4,16,"2020-08-30"),(13,791,4,9,28,"2020-08-31"),
        (14,983,1,34,61,"2020-08-01"),(14,999,1,78,32,"2020-08-02"),(14,1039,0,34,19,"2020-08-03"),(14,772,1,53,31,"2020-08-04"),
        (14,1023,0,29,27,"2020-08-05"),(14,983,0,60,31,"2020-08-06"),(14,1324,0,28,29,"2020-08-07"),(14,861,3,49,30,"2020-08-08"),
        (14,1034,0,38,42,"2020-08-09"),(14,836,0,0,46,"2020-08-10"),(14,927,0,35,44,"2020-08-11"),(14,697,2,26,8,"2020-08-12"),
        (14,802,0,55,12,"2020-08-13"),(14,782,0,13,34,"2020-08-14"),(14,915,1,50,18,"2020-08-15"),(14,810,0,20,17,"2020-08-16"),
        (14,709,2,71,22,"2020-08-17"),(14,888,2,12,19,"2020-08-18"),(14,827,0,38,33,"2020-08-19"),(14,1293,0,9,29,"2020-08-20"),
        (14,1823,0,59,0,"2020-08-21"),(14,1083,0,23,12,"2020-08-22"),(14,1381,1,29,19,"2020-08-23"),(14,1173,1,13,39,"2020-08-24"),
        (14,1432,0,24,47,"2020-08-25"),(14,1063,0,42,19,"2020-08-26"),(14,1402,0,33,59,"2020-08-27"),(14,929,0,13,74,"2020-08-28"),
        (14,1304,0,14,59,"2020-08-29"),(14,1145,3,33,30,"2020-08-30"),(14,1102,0,72,0,"2020-08-31"),
        (15,539,3,13,26,"2020-08-01"),(15,617,0,27,14,"2020-08-02"),(15,729,0,18,10,"2020-08-03"),(15,521,1,0,24,"2020-08-04"),
        (15,491,0,14,22,"2020-08-05"),(15,719,4,7,27,"2020-08-06"),(15,821,0,8,11,"2020-08-07"),(15,594,1,32,7,"2020-08-08"),
        (15,928,4,8,16,"2020-08-09"),(15,830,3,0,16,"2020-08-10"),(15,902,3,0,6,"2020-08-11"),(15,492,4,22,12,"2020-08-12"),
        (15,619,6,9,14,"2020-08-13"),(15,715,2,25,2,"2020-08-14"),(15,762,5,12,1,"2020-08-15"),(15,378,7,3,24,"2020-08-16"),
        (15,729,8,17,0,"2020-08-17"),(15,785,6,4,8,"2020-08-18"),(15,681,12,17,27,"2020-08-19"),(15,402,5,8,2,"2020-08-20"),
        (15,800,2,0,6,"2020-08-21"),(15,516,8,0,0,"2020-08-22"),(15,535,3,9,5,"2020-08-23"),(15,501,4,7,1,"2020-08-24"),
        (15,910,1,2,6,"2020-08-25"),(15,684,9,21,9,"2020-08-26"),(15,587,2,18,12,"2020-08-27"),(15,610,2,14,18,"2020-08-28"),
        (15,527,5,16,0,"2020-08-29"),(15,901,2,13,15,"2020-08-30"),(15,534,0,6,14,"2020-08-31"),
        (17,1648,0,3,0,"2020-08-01"),(17,1849,0,6,11,"2020-08-02"),(17,1453,0,12,10,"2020-08-03"),(17,921,0,13,5,"2020-08-04"),
        (17,1623,0,0,0,"2020-08-05"),(17,1632,0,0,0,"2020-08-06"),(17,1829,0,2,9,"2020-08-07"),(17,1834,1,0,5,"2020-08-08"),
        (17,2341,0,0,7,"2020-08-09"),(17,2483,2,0,8,"2020-08-10"),(17,1712,1,2,3,"2020-08-11"),(17,1749,0,12,9,"2020-08-12"),
        (17,1232,0,0,0,"2020-08-13"),(17,2013,0,2,0,"2020-08-14"),(17,1144,1,4,7,"2020-08-15"),(17,1334,2,21,0,"2020-08-16"),
        (17,1552,0,0,0,"2020-08-17"),(17,1934,0,3,5,"2020-08-18"),(17,1536,0,11,2,"2020-08-19"),(17,1346,0,3,1,"2020-08-20"),
        (17,920,1,8,6,"2020-08-21"),(17,1235,1,10,2,"2020-08-22"),(17,2184,1,13,8,"2020-08-23"),(17,1765,3,6,8,"2020-08-24"),
        (17,1928,0,5,4,"2020-08-25"),(17,1321,0,5,1,"2020-08-26"),(17,2562,0,0,18,"2020-08-27"),(17,1113,0,11,11,"2020-08-28"),
        (17,1057,0,3,0,"2020-08-29"),(17,1022,0,0,7,"2020-08-30"),(17,1240,0,0,0,"2020-08-31"),
        (18,782,0,46,54,"2020-08-01"),(18,492,6,40,31,"2020-08-02"),(18,362,1,49,28,"2020-08-03"),(18,593,7,12,34,"2020-08-04"),
        (18,892,2,24,62,"2020-08-05"),(18,584,3,4,52,"2020-08-06"),(18,392,0,21,35,"2020-08-07"),(18,793,1,29,29,"2020-08-08"),
        (18,1639,4,32,23,"2020-08-09"),(18,493,0,62,14,"2020-08-10"),(18,418,4,44,49,"2020-08-11"),(18,632,0,85,24,"2020-08-12"),
        (18,622,1,18,27,"2020-08-13"),(18,302,4,21,19,"2020-08-14"),(18,472,2,20,38,"2020-08-15"),(18,535,5,37,19,"2020-08-16"),
        (18,522,5,9,49,"2020-08-17"),(18,592,7,42,35,"2020-08-18"),(18,520,3,19,37,"2020-08-19"),(18,589,5,20,29,"2020-08-20"),
        (18,482,6,19,18,"2020-08-21"),(18,857,4,19,72,"2020-08-22"),(18,539,8,4,13,"2020-08-23"),(18,470,1,44,31,"2020-08-24"),
        (18,527,2,49,38,"2020-08-25"),(18,496,0,23,32,"2020-08-26"),(18,611,6,39,19,"2020-08-27"),(18,593,2,54,40,"2020-08-28"),
        (18,812,0,75,21,"2020-08-29"),(18,523,2,29,42,"2020-08-30"),(18,389,0,82,14,"2020-08-31"),
        (19,1345,1,18,21,"2020-08-01"),(19,1520,3,20,18,"2020-08-02"),(19,1037,4,19,12,"2020-08-03"),(19,1628,0,18,11,"2020-08-04"),
        (19,1129,0,14,7,"2020-08-05"),(19,1045,0,19,19,"2020-08-06"),(19,1122,3,14,15,"2020-08-07"),(19,1072,1,23,14,"2020-08-08"),
        (19,1249,2,19,17,"2020-08-09"),(19,1294,4,14,16,"2020-08-10"),(19,972,0,20,13,"2020-08-11"),(19,882,0,19,8,"2020-08-12"),
        (19,938,2,20,8,"2020-08-13"),(19,1183,0,9,8,"2020-08-14"),(19,927,2,28,19,"2020-08-15"),(19,926,1,10,7,"2020-08-16"),
        (19,910,0,11,16,"2020-08-17"),(19,1009,3,15,20,"2020-08-18"),(19,899,0,7,9,"2020-08-19"),(19,935,4,12,19,"2020-08-20"),
        (19,888,0,12,22,"2020-08-21"),(19,1212,2,11,18,"2020-08-22"),(19,1322,3,11,15,"2020-08-23"),(19,1088,2,6,13,"2020-08-24"),
        (19,1420,4,21,29,"2020-08-25"),(19,988,0,17,9,"2020-08-26"),(19,1129,0,8,10,"2020-08-27"),(19,1333,0,17,16,"2020-08-28"),
        (19,1333,1,27,19,"2020-08-29"),(19,892,1,10,4,"2020-08-30"),(19,1341,2,13,9,"2020-08-31"),
        (21,1483,0,53,34,"2020-08-01"),(21,1938,0,34,42,"2020-08-02"),(21,1305,0,78,22,"2020-08-03"),(21,1324,0,25,39,"2020-08-04"),
        (21,1837,0,32,28,"2020-08-05"),(21,2183,0,21,38,"2020-08-06"),(21,1735,1,26,43,"2020-08-07"),(21,1253,3,41,19,"2020-08-08"),
        (21,983,1,35,20,"2020-08-09"),(21,1634,3,17,29,"2020-08-10"),(21,1538,0,33,19,"2020-08-11"),(21,1653,0,55,67,"2020-08-12"),
        (21,1038,0,21,17,"2020-08-13"),(21,826,0,20,17,"2020-08-14"),(21,1492,1,82,34,"2020-08-15"),(21,1403,0,31,56,"2020-08-16"),
        (21,1392,0,27,19,"2020-08-17"),(21,1542,0,17,34,"2020-08-18"),(21,937,0,77,40,"2020-08-19"),(21,1839,0,20,42,"2020-08-20"),
        (21,1837,0,31,30,"2020-08-21"),(21,1639,0,32,46,"2020-08-22"),(21,827,1,23,37,"2020-08-23"),(21,1432,1,45,39,"2020-08-24"),
        (21,803,0,14,52,"2020-08-25"),(21,1329,0,82,52,"2020-08-26"),(21,1748,0,81,12,"2020-08-27"),(21,1522,1,50,50,"2020-08-28"),
        (21,1432,2,39,42,"2020-08-29"),(21,1238,1,68,51,"2020-08-30"),(21,2242,0,54,32,"2020-08-31"),
        (22,2403,0,52,43,"2020-08-01"),(22,2912,0,61,30,"2020-08-02"),(22,1098,1,83,21,"2020-08-03"),(22,2183,2,61,39,"2020-08-04"),
        (22,2421,0,19,37,"2020-08-05"),(22,2182,1,18,29,"2020-08-06"),(22,1128,0,66,55,"2020-08-07"),(22,2382,0,53,48,"2020-08-08"),
        (22,2018,0,22,39,"2020-08-09"),(22,1592,0,28,27,"2020-08-10"),(22,829,0,47,30,"2020-08-11"),(22,2110,0,20,20,"2020-08-12"),
        (22,1732,0,49,28,"2020-08-13"),(22,1037,0,68,21,"2020-08-14"),(22,1638,0,29,17,"2020-08-15"),(22,1044,0,16,44,"2020-08-16"),
        (22,1731,1,8,22,"2020-08-17"),(22,1111,0,72,19,"2020-08-18"),(22,1543,0,35,39,"2020-08-19"),(22,937,0,59,21,"2020-08-20"),
        (22,1372,0,3,53,"2020-08-21"),(22,1762,0,18,35,"2020-08-22"),(22,1472,0,13,28,"2020-08-23"),(22,2493,0,20,46,"2020-08-24"),
        (22,1138,0,13,42,"2020-08-25"),(22,1662,0,9,13,"2020-08-26"),(22,1600,0,95,8,"2020-08-27"),(22,891,0,39,53,"2020-08-28"),
        (22,1240,0,72,31,"2020-08-29"),(22,1144,0,23,38,"2020-08-30"),(22,1593,1,52,29,"2020-08-31"),
        (23,222,2,50,55,"2020-08-01"),(23,62,0,60,32,"2020-08-02"),(23,26,0,30,22,"2020-08-03"),(23,29,1,7,67,"2020-08-04"),
        (23,169,1,62,42,"2020-08-05"),(23,173,0,47,51,"2020-08-06"),(23,135,0,18,48,"2020-08-07"),(23,99,0,50,15,"2020-08-08"),
        (23,82,0,38,46,"2020-08-09"),(23,200,0,23,30,"2020-08-10"),(23,147,0,21,49,"2020-08-11"),(23,178,0,42,21,"2020-08-12"),
        (23,183,0,30,51,"2020-08-13"),(23,144,0,28,36,"2020-08-14"),(23,168,0,30,30,"2020-08-15"),(23,299,2,30,8,"2020-08-16"),
        (23,283,0,41,29,"2020-08-17"),(23,152,2,19,45,"2020-08-18"),(23,291,0,34,31,"2020-08-19"),(23,219,1,18,19,"2020-08-20"),
        (23,212,0,39,44,"2020-08-21"),(23,169,0,29,39,"2020-08-22"),(23,53,0,29,34,"2020-08-23"),(23,178,0,40,49,"2020-08-24"),
        (23,192,1,50,28,"2020-08-25"),(23,98,0,25,45,"2020-08-26"),(23,22,1,32,85,"2020-08-27"),(23,155,0,33,51,"2020-08-28"),
        (23,73,0,40,46,"2020-08-29"),(23,54,1,14,25,"2020-08-30"),(23,35,3,12,90,"2020-08-31"),
        (25,1839,4,42,19,"2020-08-01"),(25,1555,0,60,6,"2020-08-02"),(25,1412,0,21,7,"2020-08-03"),(25,1242,5,29,14,"2020-08-04"),
        (25,1537,2,32,17,"2020-08-05"),(25,1492,1,32,0,"2020-08-06"),(25,1500,1,27,29,"2020-08-07"),(25,1039,2,37,28,"2020-08-08"),
        (25,1392,0,59,8,"2020-08-09"),(25,1637,0,43,12,"2020-08-10"),(25,902,1,22,49,"2020-08-11"),(25,1183,0,81,8,"2020-08-12"),
        (25,892,3,24,13,"2020-08-13"),(25,1228,3,50,32,"2020-08-14"),(25,1182,4,38,7,"2020-08-15"),(25,1340,0,70,21,"2020-08-16"),
        (25,989,0,9,17,"2020-08-17"),(25,1447,0,72,12,"2020-08-18"),(25,1632,0,83,11,"2020-08-19"),(25,1483,3,63,18,"2020-08-20"),
        (25,1892,4,47,20,"2020-08-21"),(25,963,0,52,27,"2020-08-22"),(25,1701,0,28,17,"2020-08-23"),(25,1204,4,57,19,"2020-08-24"),
        (25,792,1,56,19,"2020-08-25"),(25,1284,0,43,12,"2020-08-26"),(25,1132,5,32,19,"2020-08-27"),(25,1874,1,48,2,"2020-08-28"),
        (25,1638,0,72,11,"2020-08-29"),(25,1195,4,50,22,"2020-08-30"),(25,1233,2,58,4,"2020-08-31"),
        (26,412,5,59,42,"2020-08-01"),(26,298,5,77,45,"2020-08-02"),(26,429,7,34,47,"2020-08-03"),(26,404,8,44,60,"2020-08-04"),
        (26,383,13,38,30,"2020-08-05"),(26,288,7,61,15,"2020-08-06"),(26,502,4,17,50,"2020-08-07"),(26,593,6,29,36,"2020-08-08"),
        (26,492,9,28,24,"2020-08-09"),(26,232,13,21,41,"2020-08-10"),(26,489,3,20,29,"2020-08-11"),(26,605,4,39,52,"2020-08-12"),
        (26,502,2,35,41,"2020-08-13"),(26,399,15,23,33,"2020-08-14"),(26,400,3,40,33,"2020-08-15"),(26,472,13,35,49,"2020-08-16"),
        (26,664,8,18,33,"2020-08-17"),(26,477,6,41,38,"2020-08-18"),(26,302,7,67,28,"2020-08-19"),(26,533,2,58,30,"2020-08-20"),
        (26,544,11,55,25,"2020-08-21"),(26,425,3,57,29,"2020-08-22"),(26,239,14,50,33,"2020-08-23"),(26,421,6,62,43,"2020-08-24"),
        (26,372,13,39,59,"2020-08-25"),(26,362,1,14,40,"2020-08-26"),(26,277,3,60,48,"2020-08-27"),(26,388,10,50,50,"2020-08-28"),
        (26,301,8,78,30,"2020-08-29"),(26,311,9,92,3,"2020-08-30"),(26,399,2,67,21,"2020-08-31"),
        (27,183,5,15,37,"2020-08-01"),(27,283,6,3,29,"2020-08-02"),(27,156,9,33,25,"2020-08-03"),(27,253,14,8,39,"2020-08-04"),
        (27,200,7,22,28,"2020-08-05"),(27,109,8,16,21,"2020-08-06"),(27,161,7,18,48,"2020-08-07"),(27,200,5,12,62,"2020-08-08"),
        (27,126,4,13,31,"2020-08-09"),(27,193,0,7,36,"2020-08-10"),(27,98,5,16,53,"2020-08-11"),(27,121,3,17,30,"2020-08-12"),
        (27,255,9,28,20,"2020-08-13"),(27,244,9,37,24,"2020-08-14"),(27,188,11,21,37,"2020-08-15"),(27,249,6,24,24,"2020-08-16"),
        (27,242,1,31,23,"2020-08-17"),(27,237,15,15,57,"2020-08-18"),(27,264,6,7,54,"2020-08-19"),(27,255,1,41,19,"2020-08-20"),
        (27,251,13,7,44,"2020-08-21"),(27,281,2,30,34,"2020-08-22"),(27,338,3,23,43,"2020-08-23"),(27,201,13,8,57,"2020-08-24"),
        (27,277,11,13,38,"2020-08-25"),(27,299,12,28,19,"2020-08-26"),(27,139,1,22,47,"2020-08-27"),(27,267,10,16,40,"2020-08-28"),
        (27,178,8,19,27,"2020-08-29"),(27,177,17,9,45,"2020-08-30"),(27,87,12,11,18,"2020-08-31"),
        (28,1124,14,5,12,"2020-08-01"),(28,1044,7,7,7,"2020-08-02"),(28,755,13,6,6,"2020-08-03"),(28,1533,18,7,2,"2020-08-04"),
        (28,679,18,8,4,"2020-08-05"),(28,677,11,1,8,"2020-08-06"),(28,632,16,7,0,"2020-08-07"),(28,1034,7,3,3,"2020-08-08"),
        (28,575,2,14,4,"2020-08-09"),(28,732,15,25,0,"2020-08-10"),(28,687,2,3,15,"2020-08-11"),(28,543,9,11,16,"2020-08-12"),
        (28,533,7,4,9,"2020-08-13"),(28,789,4,19,17,"2020-08-14"),(28,555,1,14,9,"2020-08-15"),(28,566,12,13,27,"2020-08-16"),
        (28,423,14,3,5,"2020-08-17"),(28,643,9,7,2,"2020-08-18"),(28,502,7,5,5,"2020-08-19"),(28,624,13,1,17,"2020-08-20"),
        (28,1463,5,0,11,"2020-08-21"),(28,611,12,8,11,"2020-08-22"),(28,487,14,3,16,"2020-08-23"),(28,620,8,22,0,"2020-08-24"),
        (28,699,11,14,8,"2020-08-25"),(28,586,10,17,0,"2020-08-26"),(28,679,9,0,4,"2020-08-27"),(28,1111,5,9,6,"2020-08-28"),
        (28,581,8,21,10,"2020-08-29"),(28,581,6,4,13,"2020-08-30"),(28,700,6,7,13,"2020-08-31"),
        (29,1524,0,56,25,"2020-08-01"),(29,987,0,42,39,"2020-08-02"),(29,1439,1,34,38,"2020-08-03"),(29,1188,0,41,37,"2020-08-04"),
        (29,1730,0,48,39,"2020-08-05"),(29,1771,0,49,43,"2020-08-06"),(29,1378,0,41,29,"2020-08-07"),(29,1263,0,33,59,"2020-08-08"),
        (29,1930,1,27,57,"2020-08-09"),(29,1849,1,38,36,"2020-08-10"),(29,1598,0,39,37,"2020-08-11"),(29,1730,0,21,68,"2020-08-12"),
        (29,1847,0,64,19,"2020-08-13"),(29,1667,0,30,68,"2020-08-14"),(29,1702,0,68,31,"2020-08-15"),(29,2740,0,28,46,"2020-08-16"),
        (29,1222,0,30,34,"2020-08-17"),(29,1493,4,27,59,"2020-08-18"),(29,1832,2,84,18,"2020-08-19"),(29,2489,0,34,32,"2020-08-20"),
        (29,1352,2,74,40,"2020-08-21"),(29,1339,0,18,22,"2020-08-22"),(29,1440,0,36,36,"2020-08-23"),(29,2009,2,50,56,"2020-08-24"),
        (29,1667,0,55,36,"2020-08-25"),(29,1028,0,37,65,"2020-08-26"),(29,927,0,25,39,"2020-08-27"),(29,2021,0,33,13,"2020-08-28"),
        (29,1599,0,61,38,"2020-08-29"),(29,1088,0,70,39,"2020-08-30"),(29,1073,1,47,27,"2020-08-31"),
        (30,329,2,20,45,"2020-08-01"),(30,222,8,10,47,"2020-08-02"),(30,173,12,4,41,"2020-08-03"),(30,177,13,7,52,"2020-08-04"),
        (30,710,6,10,42,"2020-08-05"),(30,334,2,2,37,"2020-08-06"),(30,123,16,8,39,"2020-08-07"),(30,189,15,8,46,"2020-08-08"),
        (30,400,5,7,29,"2020-08-09"),(30,354,6,20,78,"2020-08-10"),(30,394,9,6,79,"2020-08-11"),(30,207,9,11,83,"2020-08-12"),
        (30,329,0,4,47,"2020-08-13"),(30,378,6,4,44,"2020-08-14"),(30,338,14,7,39,"2020-08-15"),(30,284,2,6,93,"2020-08-16"),
        (30,284,0,6,36,"2020-08-17"),(30,312,13,1,39,"2020-08-18"),(30,267,17,6,29,"2020-08-19"),(30,351,8,15,43,"2020-08-20"),
        (30,244,4,0,67,"2020-08-21"),(30,200,4,14,29,"2020-08-22"),(30,313,11,12,58,"2020-08-23"),(30,273,14,14,55,"2020-08-24"),
        (30,285,12,11,39,"2020-08-25"),(30,173,5,9,55,"2020-08-26"),(30,461,10,13,64,"2020-08-27"),(30,290,17,29,61,"2020-08-28"),
        (30,197,7,8,51,"2020-08-29"),(30,183,15,4,64,"2020-08-30"),(30,245,16,50,43,"2020-08-31")]

df_consumos_diarios = spark.createDataFrame(data, columns)
df_consumos_diarios.write.mode("overwrite").option("encoding", "UTF-8").saveAsTable('df_consumos_diarios')
#display(df_consumos_diarios)



# Para escribir una tabla de databricks no es necesario especificar la base de datos porque se guardan en la BBDD por defecto (default).
# Trabajando desde el ecosistema Cloudera, es necesario especificar: saveAsTable("database_name.a_consumos")
#df_consumos_diarios.write.mode("overwrite").saveAsTable('a_consumos')


# Escribir dataframe como CSV en el filesystem de Databricks
#df_consumos_diarios.write.format('com.databricks.spark.csv').save('dbfs:/FileStore/imf_big_data',header='true')

**1. Mostrar por pantalla el número TOTAL de clientes que tenían en el mes anterior más de un contrato con la compañía**

In [0]:
df1=spark.sql("select id_cliente, count(*) Total from df_facturas_mes_ant group by id_cliente" )
display(df1)

id_cliente,Total
26,1
29,1
22,2
7,1
25,2
6,1
27,2
17,1
5,2
1,3


In [0]:
print("el numero de personas con mas de una factura el anterior mes es de " + str(df1.filter(df1['Total']>1).count()))

el numero de personas con mas de una factura el anterior mes es de 8

**2. Generar un nuevo dataframe de facturas del mes actual que asigne un 7% de descuento a todos los contratos de clientes que ya existían en el mes anterior (los contratos de los clientes nuevos seguirán con el mismo importe). Mantener la columna "importe" y crear una nueva columna "importe_dto" con el nuevo importe, casteada a 2 decimales. Mostrar además el resultado ordenado por los campos id_cliente ascendente, importe descendente. **

In [0]:
df_descuento_flg = df_facturas_mes_actual.join(df_facturas_mes_ant.select("id_cliente").withColumn("flg_mes_ant", lit(1)),["id_cliente"],"left") \
                                                                            .dropDuplicates(["id_cliente","id_oferta","importe"]).orderBy("id_cliente")

In [0]:
df_descuento = df_descuento_flg.withColumn('importe_dto', round( when( col('flg_mes_ant') >0, df_facturas_mes_actual.importe * 0.93).otherwise (df_facturas_mes_actual.importe), 2 ) ).orderBy("id_cliente", col('importe').desc())
display(df_descuento)

id_cliente,id_oferta,importe,fecha,flg_mes_ant,importe_dto
1,6,124.5,2020-08-31,1,115.79
1,6,118.6,2020-08-31,1,110.3
2,4,95.0,2020-08-31,1,88.35
3,5,102.5,2020-08-31,1,95.33
4,1,70.0,2020-08-31,null,70.0
5,7,80.5,2020-08-31,1,74.87
5,9,40.5,2020-08-31,1,37.67
6,8,84.0,2020-08-31,1,78.12
7,4,95.0,2020-08-31,1,88.35
8,5,100.5,2020-08-31,1,93.47


**3. Por problemas de saturación en la red debido al incremento en el uso de datos por el confinamiento debido a la COVID-19, se decide limitar el uso de datos este mes subiendo la tarifa de todas las ofertas que incluyan "datos ilimitados" en un 15%. Obtener un DF con las mismas columnas que el DF de facturas del mes actual y a mayores la columna "importe_dto"**.

In [0]:
df_temporal = df_facturas_mes_actual.join(df_ofertas.select("id_oferta").withColumn("importe_dto", lit(1)),["id_oferta"],"left") \
                                                                            .dropDuplicates(["id_cliente","id_oferta","importe"]).orderBy("id_cliente")

In [0]:
df_caro = df_facturas_mes_actual.withColumn('importe_dto', when ((col('id_oferta') == 3)| (col('id_oferta') == 4) |(col('id_oferta') == 5) | (col('id_oferta')== 6) |(col('id_oferta') == 9) , df_facturas_mes_actual.importe*1.15).otherwise(df_facturas_mes_actual.importe))
display(df_caro)

id_cliente,id_oferta,importe,fecha,importe_dto
1,6,124.5,2020-08-31,143.17499999999998
1,6,118.6,2020-08-31,136.39
2,4,95.0,2020-08-31,109.24999999999999
3,5,102.5,2020-08-31,117.87499999999999
4,1,70.0,2020-08-31,70.0
5,7,80.5,2020-08-31,80.5
5,9,40.5,2020-08-31,46.574999999999996
6,8,84.0,2020-08-31,84.0
7,4,95.0,2020-08-31,109.24999999999999
8,5,100.5,2020-08-31,115.57499999999999


**4. Crear una nueva variable "grupo_edad" que agrupe a los clientes, tanto del mes_actual como del mes_anterior en 4 rangos según su edad asignando valores del 1 - 4 segun el rango al que pertenezcan (18-25 (1), 26-40 (2), 41-65 (3), >65 (4)). Obtener una tabla resumen que extraiga para cada para cada uno de los 4 grupos identificados la MEDIA de consumo de datos, sms enviados, minutos_movil, minutos_fijo, con todos los campos casteados a 2 decimales y ordenar el DF por grupo_edad ascendente. Extraer conclusiones.**

In [0]:
df_clientes_join = df_clientes.select("id_cliente","edad")

In [0]:
df_facturas_mes_ante = df_facturas_mes_ant.join(df_clientes_join,["id_cliente"],"left").orderBy("id_cliente")
df_facturas_mes_actuali = df_facturas_mes_actual.join(df_clientes_join,["id_cliente"],"left").orderBy("id_cliente")


In [0]:
df_facturas_mes_actual_flg = df_facturas_mes_actual.join(df_facturas_mes_ant.select("id_cliente").withColumn("flg_mes_ant", lit(1)),["id_cliente"],"left") \
                                                                            .dropDuplicates(["id_cliente","id_oferta","importe"]).orderBy("id_cliente")

df_facturas_mes_actual_flg.show()             

+----------+---------+-------+----------+-----------+
id_cliente|id_oferta|importe| fecha|flg_mes_ant|
+----------+---------+-------+----------+-----------+
 1| 6| 118.6|2020-08-31| 1|
 1| 6| 124.5|2020-08-31| 1|
 2| 4| 95.0|2020-08-31| 1|
 3| 5| 102.5|2020-08-31| 1|
 4| 1| 70.0|2020-08-31| null|
 5| 7| 80.5|2020-08-31| 1|
 5| 9| 40.5|2020-08-31| 1|
 6| 8| 84.0|2020-08-31| 1|
 7| 4| 95.0|2020-08-31| 1|
 8| 5| 100.5|2020-08-31| 1|
 9| 7| 78.5|2020-08-31| null|
 9| 10| 29.99|2020-08-31| null|
 10| 6| 124.5|2020-08-31| 1|
 11| 3| 83.0|2020-08-31| 1|
 12| 1| 70.0|2020-08-31| 1|
 13| 4| 95.0|2020-08-31| 1|
 14| 9| 42.5|2020-08-31| 1|
 15| 2| 78.5|2020-08-31| 1|
 15| 6| 115.0|2020-08-31| 1|
 17| 8| 84.0|2020-08-31| 1|
+----------+---------+-------+----------+-----------+
only showing top 20 rows

In [0]:
df_facturas_mes_ante = df_facturas_mes_ante.withColumn("grupo_edad", when(col("edad") < 26, lit(1))
                                         .when(col("edad").between(26,40),lit(2))
                                         .when(col("edad").between(41,65),lit(3))
                                         .otherwise(lit(4)))

df_facturas_mes_actuali = df_facturas_mes_actuali.withColumn("grupo_edad", when(col("edad") < 26, lit(1))
                                         .when(col("edad").between(26,40),lit(2))
                                         .when(col("edad").between(41,65),lit(3))
                                         .otherwise(lit(4)))

df_facturas_mes_actuali.show()

+----------+---------+-------+----------+----+----------+
id_cliente|id_oferta|importe| fecha|edad|grupo_edad|
+----------+---------+-------+----------+----+----------+
 1| 6| 124.5|2020-08-31| 26| 2|
 1| 6| 118.6|2020-08-31| 26| 2|
 2| 4| 95.0|2020-08-31| 58| 3|
 3| 5| 102.5|2020-08-31| 68| 4|
 4| 1| 70.0|2020-08-31| 32| 2|
 5| 7| 80.5|2020-08-31| 41| 3|
 5| 9| 40.5|2020-08-31| 41| 3|
 6| 8| 84.0|2020-08-31| 74| 4|
 7| 4| 95.0|2020-08-31| 19| 1|
 8| 5| 100.5|2020-08-31| 49| 3|
 9| 10| 29.99|2020-08-31| 24| 1|
 9| 7| 78.5|2020-08-31| 24| 1|
 10| 6| 124.5|2020-08-31| 40| 2|
 11| 3| 83.0|2020-08-31| 20| 1|
 12| 1| 70.0|2020-08-31| 75| 4|
 13| 4| 95.0|2020-08-31| 53| 3|
 14| 9| 42.5|2020-08-31| 30| 2|
 15| 2| 78.5|2020-08-31| 52| 3|
 15| 6| 115.0|2020-08-31| 52| 3|
 17| 8| 84.0|2020-08-31| 18| 1|
+----------+---------+-------+----------+----+----------+
only showing top 20 rows

**5. Se quiere realizar un estudio por Sexo para analizar si son las mujeres o los hombres quiénes consumen más datos durante el fin de semana y hacen más llamadas desde el móvil. Para ello se deberá, sin ayuda de un calendario, extraer el día de la semana al que corresponde cada una de las fechas del mes de
Agosto para saber cuáles son fin de semana (se consideran días de fin de semana el viernes, sábado y domingo). </br> El DF a obtener deberá tener 2 registros con las columnas: sexo, total_mins_movil_finde, total_datos_moviles_finde. </br> Extraer conclusiones tras presentar el DF resultante.**

In [0]:
def findDay(date): 
    born = datetime.datetime.strptime(date, '%d %m %Y').weekday() 
    return (calendar.day_name[born]) 

In [0]:
#df_registros_fds = df_consumos_diarios.withColumn('quediafds', findDay(df_consumos_diarios.fecha))

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-1890334010118092> in <module> 
 ----> 1 df_registros_fds = df_consumos_diarios . withColumn ( 'quediafds' , findDay ( df_consumos_diarios . fecha ) ) 

 <command-1262156798945043> in findDay (date) 
 1 def findDay ( date ) : 
 ----> 2 born = datetime . datetime . strptime ( date , '%d %m %Y' ) . weekday ( ) 
 3 return ( calendar . day_name [ born ] ) 

 TypeError : strptime() argument 1 must be str, not Column

In [0]:
%sql
select C.sexo , sum( D.minutos_llamadas_movil)  total_mins_movil_finde, sum( D.consumo_datos_MB) total_datos_moviles_finde from df_consumos_diarios D, df_clientes C  where  D.id_cliente=C.id_cliente and  (dayofweek(D.fecha) > 5 or dayofweek(D.fecha) < 2) group by c.sexo

sexo,total_mins_movil_finde,total_datos_moviles_finde
F,7222,148009
M,3459,139267


**6. Obtener un DF que contenga 4 registros, que serán el cliente de cada grupo edad que más datos móviles ha consumido durante los 15 primeros días del mes de Agosto (día 15 incluido en el cálculo). El DF deberá contener las columnas: nombre, edad, grupo_edad, datos_moviles_total_15, max_sms_enviados_15** </br> (max_sms_enviados_15 contiene el máximo de sms enviados en un día por el cliente con más datos_moviles consumidos de cada grupo_edad durante esos 15 primeros días del mes). </br>
**Extraer conclusiones en cuanto a datos consumidos y sms enviados por cada grupo_edad.**

In [0]:
df_clientes_edad = df_clientes.withColumn("grupo_edad", when(col("edad") < 26, lit(1))
                                         .when(col("edad").between(26,40),lit(2))
                                         .when(col("edad").between(41,65),lit(3))
                                         .otherwise(lit(4)))
df_clientes_edad.write.mode("overwrite").option("encoding", "UTF-8").saveAsTable('df_clientes_edades')
display(df_clientes_edad
       )

id_cliente,nombre,edad,sexo,provincia,pais,grupo_edad
1,Pablo Perez,26,M,Madrid,España,2
2,Eduardo Redondo,58,M,Bogota,Colombia,3
3,Roberto Salazar,68,M,Monterrey,Mexico,4
4,Pedro Conde,32,M,Madrid,España,2
5,Ana Robles,41,F,Valladolid,España,3
6,David Roldan,74,M,Guadalajara,Mexico,4
7,Carmen Arauzo,19,F,Medellin,Colombia,1
8,Silvia Rodriguez,49,F,Barcelona,España,3
9,David Cardoso,24,M,Cali,Colombia,1
10,Pablo Lopez,40,M,Sevilla,España,2


In [0]:
%sql
select  ED.grupo_edad, DI.id_cliente, max(DI.consumo_datos_MB) datos_moviles_total_15 
from df_clientes_edades ED, df_consumos_diarios DI 
where ED.id_cliente=DI.id_cliente and (DI.fecha< '2020-08-16') 
group by ED.grupo_edad, DI.id_cliente 
having max(DI.consumo_datos_MB) = (select  max( DI1.consumo_datos_MB)  
from df_clientes_edades ED1, df_consumos_diarios DI1 
where ED1.id_cliente=DI1.id_cliente and (DI1.fecha< '2020-08-16') and ED1.grupo_edad = 4)

grupo_edad,id_cliente,datos_moviles_total_15
4,30,710


In [0]:
%sql
select  max( DI1.consumo_datos_MB)  
from df_clientes_edades ED1, df_consumos_diarios DI1 
where ED1.id_cliente=DI1.id_cliente and (DI1.fecha< '2020-08-16') and ED1.grupo_edad = 1

max(consumo_datos_MB)
2912


In [0]:
%sql
select  ED.grupo_edad, DI.id_cliente, DI.consumo_datos_MB datos_moviles_total_15 
from df_clientes_edades ED, df_consumos_diarios DI 
where ED.id_cliente = DI.id_cliente and (DI.fecha< '2020-08-16') and DI.consumo_datos_MB = (select  max( DI1.consumo_datos_MB)  
from df_clientes_edades ED1, df_consumos_diarios DI1 
where ED1.id_cliente=DI1.id_cliente and (DI1.fecha< '2020-08-16') and ED1.grupo_edad = ED.grupo_edad)

grupo_edad,id_cliente,datos_moviles_total_15
4,30,710
2,21,2183
1,22,2912
3,18,1639


In [0]:
df_6=spark.sql  ( " select  ED.grupo_edad, DI.id_cliente, DI.consumo_datos_MB datos_moviles_total_15, 	(select  max( DI2.sms_enviados)  	 from df_clientes_edades ED2, df_consumos_diarios DI2 	 where ED2.id_cliente = DI2.id_cliente and DI.id_cliente = DI2.id_cliente and (DI2.fecha< '2020-08-16') and ED2.grupo_edad = ED.grupo_edad) max_sms_enviados_15 from df_clientes_edades ED, df_consumos_diarios DI where ED.id_cliente = DI.id_cliente and (DI.fecha< '2020-08-16') and DI.consumo_datos_MB = (select  max( DI1.consumo_datos_MB)  from df_clientes_edades ED1, df_consumos_diarios DI1 where ED1.id_cliente=DI1.id_cliente and (DI1.fecha< '2020-08-16') and ED1.grupo_edad = ED.grupo_edad) ORDER BY grupo_edad")
display(df_6)

grupo_edad,id_cliente,datos_moviles_total_15,max_sms_enviados_15
1,22,2912,2
2,21,2183,3
3,18,1639,7
4,30,710,16


In [0]:
df_6 = df_6.join(df_clientes, ['id_cliente'], 'left')
df_6 = df_6.drop (* ('id_cliente', 'sexo', 'provincia', 'pais'))
display(df_6)

grupo_edad,datos_moviles_total_15,max_sms_enviados_15,nombre,edad
1,2912,2,Clara Suarez,21
3,1639,7,Laura Luiz,44
2,2183,3,Fatima Cuevas,29
4,710,16,Marta Rodrigo,67


**7. Nos interesa averiguar los minutos de llamadas de los clientes que son nuevos este mes para realizar un estudio del impacto que tendrán en caso de producirse un pico de volumen de llamadas en la red. </br> Obtener un DF que contenga sólo a los clientes nuevos de este mes con 4 columnas: nombre_cliente_nuevo, edad, importe_total_mes_actual, total_minutos. </br> Se ordenará el DF resultante por la columna "total_minutos" que contiene el total de minutos de llamadas fijas y móviles durante este mes para cada cliente, ordenándolos de manera que el primer cliente nuevo que aparezca sea el que menos minutos de llamadas ha realizado en el mes de Agosto.</br>** NOTAS: Si hay solo 2 clientes nuevos, el DF a obtener tendrá sólo 2 registros. </br> Tener presente que a la hora de hacer un join, la clave de unión debe ser única en la tabla de la derecha o de otra manera se multiplicarán los registros resultantes de manera descontrolada.
</br> Tener en cuenta que si un cliente nuevo tiene 2 ofertas contratadas, la columna importe_mes_actual deberá contener la suma de los importes de ambas ofertas.

In [0]:
df_7 = spark.sql( "select M.* from df_facturas_mes_actual M left join df_facturas_mes_ant MA on M.id_cliente=MA.id_cliente where MA.id_cliente is null ")
display(df_7)

id_cliente,id_oferta,importe,fecha
28,5,102.5,2020-08-31
4,1,70.0,2020-08-31
9,7,78.5,2020-08-31
9,10,29.99,2020-08-31
19,3,85.0,2020-08-31
21,6,124.5,2020-08-31


In [0]:
df_7 = df_7.join(df_clientes.select('nombre', 'edad', 'id_cliente'), ['id_cliente'], 'left')
df_7.write.mode("overwrite").option("encoding", "UTF-8").saveAsTable('df_7')
display(df_7)

id_cliente,id_oferta,importe,fecha,nombre,edad
19,3,85.0,2020-08-31,Juan Carlos Iglesias,38
9,7,78.5,2020-08-31,David Cardoso,24
9,10,29.99,2020-08-31,David Cardoso,24
28,5,102.5,2020-08-31,Walter Ramos,54
4,1,70.0,2020-08-31,Pedro Conde,32
21,6,124.5,2020-08-31,Fatima Cuevas,29


In [0]:
df_71 = spark.sql("select d7.nombre, d7.edad, (select sum(dd1.minutos_llamadas_movil) from  df_7 d71 inner join df_consumos_diarios dd1  where d71.id_cliente = dd1.id_cliente and d71.id_cliente = d7.id_cliente  ) llamadasmovil, (select sum(dd2.minutos_llamadas_fijo) from  df_7 d72 inner join df_consumos_diarios dd2  where d72.id_cliente = dd2.id_cliente and d72.id_cliente = d7.id_cliente  ) llamadasfijo from df_7 d7 inner join df_consumos_diarios dd where d7.id_cliente = dd.id_cliente ")
df_71= df_71.dropDuplicates(["nombre","edad","llamadasmovil","llamadasfijo"])
display(df_71)

nombre,edad,llamadasmovil,llamadasfijo
Walter Ramos,54,268,260
David Cardoso,24,470,266
Fatima Cuevas,29,1264,1102
Pedro Conde,32,1630,309
Juan Carlos Iglesias,38,482,441


In [0]:
df_71=df_71.withColumn('total_minutos', col("llamadasmovil") + col("llamadasfijo")).drop("llamadasmovil" , "llamadasfijo").sort("total_minutos")
display(df_71)

nombre,edad,total_minutos
Walter Ramos,54,528
David Cardoso,24,736
Juan Carlos Iglesias,38,923
Pedro Conde,32,1939
Fatima Cuevas,29,2366


**8. Obtener un DF que contenga, para los clientes que ya existían en el mes anterior y siguen dados de alta este mes, 3 columnas: nombre, edad, n_dias_sin_sms. La última columna se refiere a obtener el número de veces que cada cliente NO ha enviado ningún SMS en el mes de Agosto. Forzar a que el comando "show" muestre 30 valores.** </br> (Si en todo el mes el cliente "A" no envío SMS 3 días en todo el mes, esta columna deberá contener el valor 3. Si hubiera algún cliente que hubiese enviado al menos 1 sms todos los dias del mes, tendrá valor 0 en esta columna).

In [0]:
df_8= df_facturas_mes_ant.select("id_cliente").join(df_clientes.select('nombre','edad',"id_Cliente"),["id_cliente"],"left").join(df_facturas_mes_actual.select('id_cliente'),["id_Cliente"],"inner")
df_8=df_8.dropDuplicates(["id_cliente"]).sort('id_cliente')
df_8.show()

+----------+-----------------+----+
id_cliente| nombre|edad|
+----------+-----------------+----+
 1| Pablo Perez| 26|
 2| Eduardo Redondo| 58|
 3| Roberto Salazar| 68|
 5| Ana Robles| 41|
 6| David Roldan| 74|
 7| Carmen Arauzo| 19|
 8| Silvia Rodriguez| 49|
 10| Pablo Lopez| 40|
 11| Luis Rodrigues| 20|
 12| Alvaro Monroy| 75|
 13| Victor Ruiz| 53|
 14| Melisa Aguado| 30|
 15|Cristian Cuadrado| 52|
 17| Jorge Recio| 18|
 18| Laura Luiz| 44|
 22| Clara Suarez| 21|
 23| Fernanda Gomez| 78|
 25| Ines Barcero| 29|
 26| Celia Castro| 47|
 27| Roberta Varado| 64|
+----------+-----------------+----+
only showing top 20 rows

In [0]:
df_81 = df_consumos_diarios.select('id_cliente', 'sms_enviados').join(df_8, ['id_cliente'] , 'inner')
df_81 = df_81.withColumn( 'cuanto0', when(df_81.sms_enviados == 0, lit(1)).otherwise(lit(0))).drop('id_cliente','sms_enviados')
df_81 = df_81.groupby('nombre','edad').sum().drop('sum(edad)').withColumnRenamed('sum(cuanto0)',' n_dias_sin_sms')
df_81.show(n=30, truncate=False)

+-----------------+----+---------------+
nombre |edad| n_dias_sin_sms|
+-----------------+----+---------------+
Pablo Perez |26 |14 |
Celia Castro |47 |0 |
Romina Verde |29 |23 |
David Roldan |74 |1 |
Ana Robles |41 |3 |
Silvia Rodriguez |49 |2 |
Luis Rodrigues |20 |21 |
Jorge Recio |18 |22 |
Clara Suarez |21 |26 |
Carmen Arauzo |19 |19 |
Cristian Cuadrado|52 |5 |
Ines Barcero |29 |13 |
Roberta Varado |64 |1 |
Marta Rodrigo |67 |2 |
Laura Luiz |44 |7 |
Fernanda Gomez |78 |21 |
Victor Ruiz |53 |7 |
Eduardo Redondo |58 |7 |
Roberto Salazar |68 |3 |
Alvaro Monroy |75 |3 |
Melisa Aguado |30 |20 |
Pablo Lopez |40 |11 |
+-----------------+----+---------------+

**9. Queremos obtener un Coeficiente de Ponderación que nos permita evaluar a cada cliente en función de su consumo para identificar los clientes más atractivos que forman parte de nuestra compañía. ESTE CÁLCULO SÓLO SE REALIZARÁ PARA LOS CLIENTES QUE TENGAN UNA SOLA OFERTA CONTRATADA CON LA COMPAÑÍA. </br> Este coeficiente se obtendrá en base a los consumos diarios, y por tanto sólo se tendrán en cuenta los clientes que existen en el mes de Agosto, ya que no tenemos datos de consumo del mes de Julio. </br> Las ponderaciones que se darán a cada uno de los consumos son las siguientes:** 
- 0.4 --> llamadas desde telefóno móvil.
- 0.3 --> datos_móviles_MB
- 0.2 --> llamadas desde teléfono fijo
- 0.1 --> SMS enviados </br>

**Los pasos a seguir son los siguientes:**
  1. Obtener la suma de todos los días de cada uno de los 4 consumos para cada cliente.
  2. Obtener el máximo del cálculo anterior de todos los clientes para poder obtener un valor entre 0 y 1 para cada uno de los 4 consumos (recordar que sólo aplica para los clientes con UNA SOLA OFERTA CONTRATADA en el mes de Agosto. El cliente con mayor consumo de datos tendrá un valor de 1 en la columna 'datos_móviles_0_1')
  3. Multiplicar estas columnas obtenidas con valor entre 0 y 1 por su ponderación correspondiente (por ejemplo la columna "datos_moviles_0_1" se multiplicará por la ponderación de datos móviles 0.4).
  4. Por último se calculará la suma de las 4 ponderaciones obtenidas para calcular el Coeficiente de ponderación buscado (coeficiente_cliente), casteado a 3 decimales.
  5. Mostrar DF resultante ordenado por este coeficiente en sentido descendente, de manera que el primer registro corresponderá al cliente más atractivo. **El DF resultante tendrá 3 columnas: nombre, edad, coeficiente_cliente.**

In [0]:
clientes_atractivos = df_facturas_mes_actual.select('id_cliente').groupby('id_cliente').count()
clientes_atractivos= clientes_atractivos.select('id_cliente').where(col('count')==1)
display(clientes_atractivos)

id_cliente
26
29
19
7
6
27
17
28
10
3


In [0]:
clientes_atrac = clientes_atractivos.join( df_consumos_diarios, ['id_cliente'], "left").drop('fecha')
clientes_atrac1= clientes_atrac.groupby("id_cliente").sum().drop('sum(id_cliente)').withColumnRenamed('sum(consumo_datos_MB)', 'datos_moviles' ).withColumnRenamed('sum(sms_enviados)', 'SMS_enviados' ).withColumnRenamed("sum(minutos_llamadas_movil)", 'llamadas_moviles' ).withColumnRenamed("sum(minutos_llamadas_fijo)", 'llamadas_fijo' )


#clientes_atrac1 = clientes_atrac1.withColumn('max', clientes_atrac1.select(max('datos_moviles')  ))
clientes_atrac1 = clientes_atrac1.withColumn('datos_moviles', col('datos_moviles')/ (clientes_atrac1.groupby().max('datos_moviles').first().asDict()['max(datos_moviles)']))\
.withColumn('SMS_enviados', col('SMS_enviados')/ (clientes_atrac1.groupby().max('SMS_enviados').first().asDict()['max(SMS_enviados)']))\
.withColumn('llamadas_moviles', col('llamadas_moviles')/ (clientes_atrac1.groupby().max('llamadas_moviles').first().asDict()['max(llamadas_moviles)']))\
.withColumn('llamadas_fijo', col('llamadas_fijo')/ (clientes_atrac1.groupby().max('llamadas_fijo').first().asDict()['max(llamadas_fijo)']))

clientes_atrac1 = clientes_atrac1.withColumn('datos_moviles', col('datos_moviles')* 0.3).withColumn('SMS_enviados', col('SMS_enviados')* 0.1)\
.withColumn('llamadas_moviles', col('llamadas_moviles')* 0.4).withColumn('llamadas_fijo', col('llamadas_fijo')* 0.2)

clientes_atrac1 = clientes_atrac1.withColumn('coeficiente_cliente', col('datos_moviles')+col('SMS_enviados')+col('llamadas_moviles')+col('llamadas_fijo'))
clientes_atrac1  = clientes_atrac1.drop('datos_moviles', 'SMS_enviados', 'llamadas_fijo', 'llamadas_moviles')
clientes_atractive = clientes_atrac1.join(df_clientes.select('nombre','edad','id_cliente'), ['id_cliente'], 'inner').drop('id_cliente').orderBy(col('coeficiente_cliente').desc())
#window_cliente = Window.partitionBy("id_cliente").orderBy(desc("importe"))
## Como puede haber ventanas con más de 2 elementos (por ejemplo el caso del id_cliente 1), creamos otra columna ("total_contratos") que nos informe del número maximo de la columna que acabamos de crear ("count_contratos").
#df_facturas_mes_ant = df_facturas_mes_ant.withColumn("total_contratos", max("count_contratos").over(window_cliente.rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)))
display(clientes_atractive)

coeficiente_cliente,nombre,edad
0.740460057116783,Romina Verde,29
0.6874500885836987,Fatima Cuevas,29
0.6526407684513884,Silvia Rodriguez,49
0.5952443726213513,Celia Castro,47
0.5516601372999321,Melisa Aguado,30
0.5166516385042719,Pedro Conde,32
0.4859989792181845,Carmen Arauzo,19
0.4398804064070377,Eduardo Redondo,58
0.4102013119225933,Fernanda Gomez,78
0.3852834254722331,Juan Carlos Iglesias,38


**10. Queremos averiguar la fecha en la que entre los 3 clientes que más datos consumen de cada grupo_edad llegan a un consumo de 20 GB de datos móviles. En caso de que algún grupo_edad no llegue entre los 3 clientes a 20 GB en todo el mes, se asignará "null" como valor de esta columna (Recordar que 1 GB = 1024 MB). </br> Obtener un DF que contiene 4 registros, uno para cada grupo_edad y 3 columnas: grupo_edad, fecha_20_GB, datos_moviles_total_grupo_3_clientes.** (datos_moviles_total_grupo_3_clientes representa el total de datos consumidos en MB por los 3 clientes del grupo hasta final de mes)

In [0]:
consumidores =  df_consumos_diarios.join(df_clientes_edad.select('id_cliente', 'grupo_edad'), ['id_cliente'], 'left').drop('sms_enviados','minutos_llamadas_movil','minutos_llamadas_fijo', 'fecha').groupBy('id_cliente','grupo_edad').sum('consumo_datos_MB').sort(col('grupo_edad'),col("sum(consumo_datos_MB)").desc())
#consumidores = consumidores.select(('id_cliente')).where()
#consumidores= consumidores[['id_cliente']].head(n = 3)
#consumirdores = consumidores.select(consumidores[['id_cliente']].head(n = 3)).groupBy("grupo_edad")
windowDept = Window.partitionBy("grupo_edad").orderBy(col("grupo_edad").desc())
consumidores =consumidores.withColumn("row",row_number().over(windowDept)).filter(col("row") <= 3).drop("row")

consumidores = consumidores.join(df_consumos_diarios.select('id_cliente', 'consumo_datos_MB', 'fecha' ), ['id_cliente'], 'left')
#windowDept = Window.partitionBy("grupo_edad").orderBy(col("fecha"))
#consumidores =consumidores.withColumn("poredad",row_number().over(windowDept))#.filter(col("row") <= 3).drop("row")
#windowDept = Window.partitionBy("id_cliente").orderBy(col("fecha"))
#consumidores =consumidores.withColumn("porcliente",row_number().over(windowDept))
consumidores = consumidores.groupby('fecha', 'grupo_edad').sum('consumo_datos_MB').sort("fecha", "grupo_edad").withColumnRenamed('sum(consumo_datos_MB)', 'consumo_datos')
#consumidores = consumidores.groupby('grupo_edad').sum('consumo_datos')#.when(sum('consumo_datos')<=(20*1024)).sort("fecha", "grupo_edad")
data_collect = consumidores.collect()


display(consumidores)

fecha,grupo_edad,consumo_datos
2020-08-01,1,6185
2020-08-01,2,4846
2020-08-01,3,2193
2020-08-01,4,674
2020-08-02,1,5854
2020-08-02,2,4480
2020-08-02,3,2397
2020-08-02,4,744
2020-08-03,1,4018
2020-08-03,2,4156


In [0]:
windowval = (Window.partitionBy('grupo_edad').orderBy('fecha')
             .rangeBetween(Window.unboundedPreceding, 0))
consumidores = consumidores.withColumn('cum_sum', sum('consumo_datos').over(windowval))
display(consumidores)

fecha,grupo_edad,consumo_datos,cum_sum
2020-08-01,1,6185,6185
2020-08-02,1,5854,12039
2020-08-03,1,4018,16057
2020-08-04,1,4864,20921
2020-08-05,1,5526,26447
2020-08-06,1,4818,31265
2020-08-07,1,5359,36624
2020-08-08,1,5054,41678
2020-08-09,1,5281,46959
2020-08-10,1,5058,52017


In [0]:
consumidores_objetivo = consumidores.where(col('cum_sum')>=20480)
consumidores_objetivo= consumidores_objetivo.groupby('grupo_edad').min('cum_sum')
consumo_mes= consumidores.groupby('grupo_edad').max('cum_sum')
final_df= consumo_mes.join(consumidores_objetivo, ['grupo_edad'], 'left')
display(final_df)

grupo_edad,max(cum_sum),min(cum_sum)
1,139687,20921
3,63100,22338
4,18995,null
2,136137,22340
